In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from IPython.display import display
from igraph import Graph
import requests, sys, os

# Functions

In [2]:
# search for deprecated or alternative symbols
def alias_mapping(ids, hgnc_table):
    resolved_ids = {}
    not_found = []
    for protein in tqdm(ids):
        
        for id_type in ['alias_symbol', 'prev_symbol']:
            new_symbol = hgnc_table.loc[hgnc_table[id_type].str.contains(protein), 'symbol'].values
            if len(new_symbol) > 0:
                resolved_ids[protein] = new_symbol[0]
                break
        else:
            not_found.append(protein)
    return resolved_ids, not_found

In [3]:
def check_dir(dir):
    if os.path.exists(dir) and os.path.isdir(dir):
        pass
    else:
        os.makedirs(dir)

# Data Exploration
This notebook shows how we explored and prepared data to feed into the machine learning pipeline.

We converted protein identifiers in all datasets to HGNC gene symbols using the HGNC complete set table. The tab separated file was downloaded from [HGNC Link](https://www.genenames.org/download/archive/) and saved in *.tsv format.

In [4]:
hgnc_table = pd.read_csv(
    "../data/raw/hgnc_complete_set.tsv",
      sep='\t', header=0, dtype='string'
      )
hgnc_table.head(2)

,hgnc_id,symbol,name,locus_group,locus_type,status,location,location_sortable,alias_symbol,alias_name,...,cd,lncrnadb,enzyme_id,intermediate_filament_db,rna_central_ids,lncipedia,gtrnadb,agr,mane_select,gencc
0,HGNC:5,A1BG,alpha-1-B glycoprotein,protein-coding gene,gene with protein product,Approved,19q13.43,19q13.43,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,HGNC:5,ENST00000263100.8|NM_130786.4,<NA>
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,non-coding RNA,"RNA, long non-coding",Approved,19q13.43,19q13.43,FLJ23569,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,A1BG-AS1,<NA>,HGNC:37133,<NA>,<NA>


In [5]:
hgnc_table.columns

Index(['hgnc_id', 'symbol', 'name', 'locus_group', 'locus_type', 'status',
       'location', 'location_sortable', 'alias_symbol', 'alias_name',
       'prev_symbol', 'prev_name', 'gene_group', 'gene_group_id',
       'date_approved_reserved', 'date_symbol_changed', 'date_name_changed',
       'date_modified', 'entrez_id', 'ensembl_gene_id', 'vega_id', 'ucsc_id',
       'ena', 'refseq_accession', 'ccds_id', 'uniprot_ids', 'pubmed_id',
       'mgd_id', 'rgd_id', 'lsdb', 'cosmic', 'omim_id', 'mirbase', 'homeodb',
       'snornabase', 'bioparadigms_slc', 'orphanet', 'pseudogene.org',
       'horde_id', 'merops', 'imgt', 'iuphar', 'kznf_gene_catalog',
       'mamit-trnadb', 'cd', 'lncrnadb', 'enzyme_id',
       'intermediate_filament_db', 'rna_central_ids', 'lncipedia', 'gtrnadb',
       'agr', 'mane_select', 'gencc'],
      dtype='object')

In [6]:
# create directories to store data generated throughout this pipeline
# each network will have a separated directory
networks = ['apid_huri', 'string']
parent_dir = '../data/processed/'
modules_dir = parent_dir + 'modules/'
check_dir(modules_dir)

network_dir = parent_dir + 'networks/'
directories = {}
for net in networks:
    graph_dir = network_dir + net + '/' + 'graph/'
    check_dir(graph_dir)
    directories[net] = graph_dir

# Networks

## APID-HuRI Network

### APID ID Mapping
APID provides HGNC symbols, as well as uniprot IDs

In [7]:
apid = pd.read_csv(
    "../data/raw/9606_noISI_Q2.txt",
      sep='\t', header=0, dtype={'UniprotID_A': 'string', 'UniprotID_B': 'string', 'GeneName_A':'string','GeneName_B':'string'}
      )
apid.head(2)

,InteractionID,UniprotID_A,UniprotName_A,GeneName_A,UniprotID_B,UniprotName_B,GeneName_B,ExpEvidences,Methods,Publications,3DStructures,CurationEvents
0,1205000,Q14160,SCRIB_HUMAN,SCRIB,B7Z2Y1,B7Z2Y1_HUMAN,<NA>,1,1,1,0,3
1,1205001,Q14160,SCRIB_HUMAN,SCRIB,Q14155,ARHG7_HUMAN,ARHGEF7,11,8,8,0,20


In [8]:
# all genes in apid
genenames = set(list(apid['GeneName_A'].dropna().unique()) + list(apid['GeneName_B'].dropna().unique()))
# all genes with approved HGNC symbol
hgnc_ids = list(hgnc_table['symbol'].unique())
# apid genes not found in hgnc table
missing_apid_ids = [protein for protein in genenames if protein not in hgnc_ids]

In [9]:
resolved_ids, not_found = alias_mapping(missing_apid_ids, hgnc_table)
print(f'{len(not_found)} symbols were not found.' )

  0%|          | 0/751 [00:00<?, ?it/s]

100%|██████████| 751/751 [00:06<00:00, 119.07it/s]

105 symbols were not found.


In [10]:
# Extract Uniprot IDs
a = apid[['GeneName_A', 'UniprotID_A']].rename(columns={'GeneName_A': 'GeneName', 'UniprotID_A': 'UniprotID'})
b = apid[['GeneName_B', 'UniprotID_B']].rename(columns={'GeneName_B': 'GeneName', 'UniprotID_B': 'UniprotID'})
uniprot_ids = pd.concat([a, b]).drop_duplicates()
not_found_uniprot_ids = uniprot_ids.loc[uniprot_ids.GeneName.isin(not_found)].to_dict('records')

In [11]:
# search in Uniprot IDs for missing symbols
remaining = []
for protein in tqdm(not_found_uniprot_ids):

    new_symbol = hgnc_table.loc[hgnc_table['uniprot_ids'].str.contains(protein['UniprotID']), 'symbol'].values
    if len(new_symbol) > 0:
        resolved_ids[protein['GeneName']] = new_symbol[0]
        
    else:
        remaining.append(protein['GeneName'])
        
print(f'{len(remaining)} symbols were not found.' )

100%|██████████| 108/108 [00:00<00:00, 203.60it/s]

104 symbols were not found.


In [12]:
# replace alternative IDs with Approved IDs
apid_graph = apid[['GeneName_A', 'GeneName_B']].replace(resolved_ids).dropna(how='any')

In [10]:
# remove IDs that could not be converted
apid_graph = apid_graph[
    ~apid_graph['GeneName_A'].isin(remaining)&
    ~apid_graph['GeneName_B'].isin(remaining)
    ].rename(columns={'GeneName_A': 'protein_A', 'GeneName_B': 'protein_B'})

apid_graph.head(2)

,protein_A,protein_B
1,SCRIB,ARHGEF7
2,SCRIB,NET1


### HuRI ID Conversion
HuRI provides only Ensembl gene IDs.

In [11]:
huri = pd.read_csv("../../data/raw/HuRI.tsv", sep='\t',header=None, names=['ENSG_A', 'ENSG_B'])
print(huri.shape[0])
huri.head(2)

52548


,ENSG_A,ENSG_B
0,ENSG00000000005,ENSG00000061656
1,ENSG00000000005,ENSG00000099968


In [12]:
# merge HuRI with the HGNC table using Ensembl IDs as keys
huri_hgnc = pd.merge(
    huri, 
    hgnc_table[['ensembl_gene_id', 'symbol']].set_index('ensembl_gene_id'),
    how='left',
    left_on='ENSG_A',
    right_on='ensembl_gene_id'
    )
huri_hgnc = pd.merge(
    huri_hgnc, 
    hgnc_table[['ensembl_gene_id', 'symbol']].set_index('ensembl_gene_id'),
    how='left',
    left_on='ENSG_B',
    right_on='ensembl_gene_id',
    suffixes=['_A', '_B']
    )

huri_hgnc.rename(columns={'symbol_A': 'protein_A', 'symbol_B': 'protein_B'}, inplace=True)

In [13]:
huri_graph = huri_hgnc[['protein_A', 'protein_B']].dropna()
huri_graph.head(2)

,protein_A,protein_B
0,TNMD,SPAG4
1,TNMD,BCL2L13


### APID & HuRI Graph
We merged the APID and HuRI graphs into a single APID-HuRI graph, which will be used throughout this work.

In [14]:
print('# of interactions (APID graph):', apid_graph.shape[0])
print('# of interactions (HuRI graph):', huri_graph.shape[0])
apid_huri = pd.concat([apid_graph, huri_graph]).drop_duplicates()
print('# of interactions (APID-HuRI graph):', apid_huri.shape[0])

# of interactions (APID graph): 263595
# of interactions (HuRI graph): 52225
# of interactions (APID-HuRI graph): 282394


In [15]:
apid_huri_graph = Graph.DataFrame(apid_huri, use_vids=False, directed=False)
print(apid_huri_graph.ecount())
apid_huri_graph = apid_huri_graph.simplify()
print(apid_huri_graph.ecount())

282394
260627


In [16]:
if not apid_huri_graph.is_connected():
    apid_huri_graph = apid_huri_graph.subgraph(apid_huri_graph.components()[0])

print(apid_huri_graph.is_connected())
print(apid_huri_graph.ecount())

True
260624


In [201]:
apid_huri.to_csv(directories['apid_huri'] + 'graph.csv', index=False)
apid_huri_graph.write_gml(directories['apid_huri'] + "graph.gml")

In [202]:
apid_huri_adj_matrix = apid_huri_graph.get_adjacency()
apid_huri_adj_matrix = np.array(apid_huri_adj_matrix.data)
np.save(directories['apid_huri'] + 'adjacency_matrix.npy', apid_huri_adj_matrix, allow_pickle=True, fix_imports=True)

## STRING Network
We evaluated two different STRING networks: one containing all types of interactions and other containing only physical interactions. We used the provided combined score to filter interactions by relevance.

### STRING ID mapping

In [78]:
string_ppi = pd.read_csv('../../data/raw/9606.protein.links.v11.5.txt', sep=' ')
print(string_ppi.shape[0])
display(string_ppi.head(2))

string_phys_ppi = pd.read_csv('../../data/raw/9606.protein.physical.links.v11.5.txt', sep=' ')
print(string_phys_ppi.shape[0])
string_phys_ppi.head(2)


11938498


,protein1,protein2,combined_score
0,9606.ENSP00000000233,9606.ENSP00000379496,155
1,9606.ENSP00000000233,9606.ENSP00000314067,197


1991832


,protein1,protein2,combined_score
0,9606.ENSP00000000233,9606.ENSP00000264718,156
1,9606.ENSP00000000233,9606.ENSP00000346046,177


In [79]:
# STRING provides an ID mapping table between Ensembl protein IDs and HGNC gene symbols
string_aliases = pd.read_csv('../../data/raw/9606.protein.aliases.v11.5.txt', sep='\t')
display(string_aliases.head(2))

,#string_protein_id,alias,source
0,9606.ENSP00000000233,2B6H,BLAST_UniProt_DR_PDB
1,9606.ENSP00000000233,2B6H,Ensembl_HGNC_UniProt_ID(supplied_by_UniProt)_D...


In [80]:
# we want HGNC gene symbols
string_aliases_hgnc = string_aliases.loc[string_aliases['source']=='Ensembl_HGNC', ['#string_protein_id', 'alias']].drop_duplicates()
print(string_aliases_hgnc.shape[0])
string_aliases_hgnc.head(2)

19142


,#string_protein_id,alias
32,9606.ENSP00000000233,ARF5
244,9606.ENSP00000000412,M6PR


In [81]:
not_in_hgnc = string_aliases_hgnc.loc[~string_aliases_hgnc['alias'].isin(hgnc_table['symbol'].to_list()), 'alias'].to_list()

In [82]:
alias, not_found = alias_mapping(not_in_hgnc, hgnc_table)
print(len(not_found))

  0%|          | 0/1194 [00:00<?, ?it/s]

100%|██████████| 1194/1194 [00:18<00:00, 63.94it/s]

14


In [83]:
# convert mapped alias
string_hgnc = string_aliases_hgnc.replace(alias)
string_hgnc = string_hgnc[~string_hgnc['alias'].isin(not_found)]
string_hgnc.shape[0]

19128

In [84]:
# String IDs with more than one HGNC symbol
duplicates = string_hgnc.loc[string_hgnc['#string_protein_id'].duplicated(), '#string_protein_id'].unique()
print(len(duplicates))

# remove duplicates
string_hgnc = string_hgnc.drop_duplicates(subset='#string_protein_id', keep=False)
print(string_hgnc.shape[0])
duplicates

8
19111


array(['9606.ENSP00000451768', '9606.ENSP00000469970',
       '9606.ENSP00000471017', '9606.ENSP00000472749',
       '9606.ENSP00000475814', '9606.ENSP00000478426',
       '9606.ENSP00000481184', '9606.ENSP00000481824'], dtype=object)

In [85]:
# STRING IDs are protein ensembl IDs
# The HGNC only has Ensembl gene IDs
# We will retrieve gene ID information from ensembl 
 
server = "https://rest.ensembl.org"
ext = "/lookup/id"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

ensembl = [gene.split('.')[1] for gene in duplicates]

r = requests.post(server+ext, headers=headers, json={'ids': ensembl})
if not r.ok:
    r.raise_for_status()
    sys.exit()

decoded = r.json()

# The first query returned transcript symbols
transcripts = {}
not_found = []
for k, i in zip(decoded, duplicates):
    if decoded[k]:
        transcripts[i] = decoded[k]['Parent']
    else:
        not_found.append(i)

transcripts

{'9606.ENSP00000451768': 'ENST00000595065',
 '9606.ENSP00000469970': 'ENST00000611660',
 '9606.ENSP00000471017': 'ENST00000616101',
 '9606.ENSP00000475814': 'ENST00000619721',
 '9606.ENSP00000478426': 'ENST00000598087',
 '9606.ENSP00000481184': 'ENST00000607355',
 '9606.ENSP00000481824': 'ENST00000599405'}

In [86]:
# The second query will return gene symbols
r = requests.post(server+ext, headers=headers, json={'ids': list(transcripts.values())})
if not r.ok:
    r.raise_for_status()
    sys.exit()

decoded = r.json()

genes = []
for k, i in zip(decoded, transcripts.keys()):
    if decoded[k]:
        genes.append((i, decoded[k]['Parent']))

genes = pd.DataFrame(genes, columns=['#string_protein_id', 'ensembl_gene_id'])
genes = pd.merge(genes, hgnc_table)[['#string_protein_id', 'symbol']].rename(columns={'symbol': 'alias'})
genes

,#string_protein_id,alias
0,9606.ENSP00000451768,MFRP
1,9606.ENSP00000469970,TMSB15C
2,9606.ENSP00000471017,H2AC19
3,9606.ENSP00000475814,TBC1D3D
4,9606.ENSP00000478426,MAGEA9B
5,9606.ENSP00000481184,CT45A8
6,9606.ENSP00000481824,OPN1MW3


In [87]:
# add symbols to the string-hgnc mapping table
print(string_hgnc.shape[0])
string_hgnc = pd.concat([string_hgnc, genes])
print(string_hgnc.shape[0])

19111
19118


In [88]:
# full network
string_full = pd.merge(
    string_ppi, string_hgnc, left_on='protein1', right_on='#string_protein_id', how='inner'
).rename(columns={'alias': 'protein_A'})

string_full = pd.merge(
    string_full, string_hgnc, left_on='protein2', right_on='#string_protein_id', how='inner'
).rename(columns={'alias': 'protein_B'})

print(string_ppi.shape[0])
print(string_full.shape[0])
string_full.head(2)

11938498
11740334


,protein1,protein2,combined_score,#string_protein_id_x,protein_A,#string_protein_id_y,protein_B
0,9606.ENSP00000000233,9606.ENSP00000379496,155,9606.ENSP00000000233,ARF5,9606.ENSP00000379496,PDE1C
1,9606.ENSP00000013807,9606.ENSP00000379496,255,9606.ENSP00000013807,ERCC1,9606.ENSP00000379496,PDE1C


In [89]:
# physical PPI network
string_phys = pd.merge(
    string_phys_ppi, string_hgnc, left_on='protein1', right_on='#string_protein_id', how='inner'
).rename(columns={'alias':'protein_A'})

string_phys = pd.merge(
    string_phys, string_hgnc, left_on='protein2', right_on='#string_protein_id', how='inner'
).rename(columns={'alias':'protein_B'})

print(string_phys_ppi.shape[0])
print(string_phys.shape[0])
string_phys.head(2)

1991832
1951474


,protein1,protein2,combined_score,#string_protein_id_x,protein_A,#string_protein_id_y,protein_B
0,9606.ENSP00000000233,9606.ENSP00000264718,156,9606.ENSP00000000233,ARF5,9606.ENSP00000264718,GPN1
1,9606.ENSP00000005257,9606.ENSP00000264718,156,9606.ENSP00000005257,RALA,9606.ENSP00000264718,GPN1


### STRING Graph

In [90]:
# evaluate different combined_score filters
stats = []
for i in [0, 400, 600, 700]:
    for l, g in zip(('Full PPI Graph', 'Physical PPI Graph'), (string_full, string_phys)):
        graph = Graph.DataFrame(
            g.loc[g.combined_score>i, ['protein_A', 'protein_B']],
            directed=False, use_vids=False
        )
        graph = graph.simplify()
        graph = graph.subgraph(graph.components()[0])

        stats.append({
            'filter': i,
            'graph': l,
            'nodes': graph.vcount(),
            'edges': graph.ecount(),
            'density': graph.density()
        })

stats = pd.DataFrame(stats)
stats

,filter,graph,nodes,edges,density
0,0,Full PPI Graph,18951,5840324,0.032526
1,0,Physical PPI Graph,18022,971356,0.005982
2,400,Full PPI Graph,18894,878839,0.004924
3,400,Physical PPI Graph,14364,205632,0.001993
4,600,Full PPI Graph,18197,386137,0.002332
5,600,Physical PPI Graph,11564,117529,0.001758
6,700,Full PPI Graph,16322,248709,0.001867
7,700,Physical PPI Graph,9486,80287,0.001785


We will use the combination of the Full PPI graph + combine_score > 700

It has a good balance of # of nodes and edges when compared with the Net4 graph (Picart-Armada et al 2019)

In [91]:
# build graph
string = string_full.loc[string_full.combined_score>700, ['protein_A', 'protein_B']]
string_graph = Graph.DataFrame(string, directed=False, use_vids=False)

In [92]:
print(string_graph.ecount())
string_graph = string_graph.simplify()
print(string_graph.ecount())

499404
248857


In [93]:
print(string_graph.is_connected())
string_graph = string_graph.subgraph(string_graph.components()[0])
print(string_graph.is_connected())
print(string_graph.ecount())

False
True
248709


In [203]:
string.to_csv(directories['string'] + "graph.csv", index=False)
string_graph.write_gml(directories['string'] + "graph.gml")

In [204]:
adj_matrix = string_graph.get_adjacency()
adj_matrix = np.array(adj_matrix.data)
np.save(directories['string'] + 'adjacency_matrix.npy', adj_matrix, allow_pickle=True, fix_imports=True)

# Reactome Processes

In [2]:
reactome = pd.read_csv(
    "../../data/raw/NCBI2ReactomeReactions.txt", sep='\t', header=None, dtype='string',
      names = ['NCBI_ID', 'Reactome_ID', 'URL', 'Event', 'Evidence_Code', 'Species']
      )
reactome = reactome[reactome['Species'] == 'Homo sapiens']
print(reactome.shape[0])
reactome.head(2)

112796


,NCBI_ID,Reactome_ID,URL,Event,Evidence_Code,Species
0,1,R-HSA-481007,https://reactome.org/PathwayBrowser/#/R-HSA-48...,Exocytosis of platelet alpha granule contents,TAS,Homo sapiens
1,1,R-HSA-6798748,https://reactome.org/PathwayBrowser/#/R-HSA-67...,Exocytosis of secretory granule lumen proteins,TAS,Homo sapiens


In [8]:
# convert entrez IDs (NCBI) to HGNC Gene Symbols
reactome_processes = pd.merge(
    reactome, hgnc_table[['entrez_id', 'symbol']].set_index('entrez_id'),
    left_on='NCBI_ID', right_on='entrez_id'
    ).rename(columns={'Reactome_ID': 'module_id', 'symbol': 'protein_id'})

reactome_processes = reactome_processes[['module_id', 'protein_id']]
reactome_processes.head(2)

,module_id,protein_id
0,R-HSA-481007,A1BG
1,R-HSA-6798748,A1BG


In [11]:
reactome_processes.to_csv(modules_dir + 'reactome.csv', index=False)

# Disgenet Disease Modules

In [12]:
disgenet = pd.read_csv(
    "../../data/raw/curated_gene_disease_associations.tsv", sep='\t', header=0)

print(disgenet.shape[0])
disgenet.head(2)

84038


,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.7,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.3,1.0,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.7,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.3,1.0,2015.0,2015.0,1,0,CTD_human


In [13]:
# all genes in Disgenet
disgenet_symbols = list(disgenet.geneSymbol.unique())
# all genes with approved HGNC symbol
hgnc_ids = list(hgnc_table['symbol'].unique())
# Disgenet genes not found in hgnc table
missing_disgenet_ids = [protein for protein in disgenet_symbols if protein not in hgnc_ids]

In [14]:
# search for deprecated or alternative symbols
resolved_ids, not_found = alias_mapping(missing_disgenet_ids, hgnc_table)
print(f'{len(not_found)} symbols were not found.' )

100%|██████████| 146/146 [00:01<00:00, 92.94it/s]

60 symbols were not found.


In [16]:
# replace alternative IDs with Approved IDs
disgenet_modules= disgenet.copy()
disgenet_modules['geneSymbol'] = disgenet_modules.geneSymbol.replace(resolved_ids)

# remove IDs that could not be converted
disgenet_modules = disgenet_modules.loc[
    ~disgenet_modules['geneSymbol'].isin(not_found),
    ['geneSymbol', 'diseaseId', 'score']
    ].rename(columns={'diseaseId': 'module_id', 'geneSymbol': 'protein_id'})
print(disgenet_modules.shape[0])
disgenet_modules.head(2)

83953


,protein_id,module_id,score
0,A1BG,C0019209,0.3
1,A1BG,C0036341,0.3


In [17]:
disgenet_modules.to_csv(modules_dir + 'disgenet.csv', index=False)